In [214]:
%run partialOrder.ipynb

..........................................................................................
----------------------------------------------------------------------
Ran 90 tests in 0.714s

OK
..........................................................................................
----------------------------------------------------------------------
Ran 90 tests in 0.727s

OK
..........................................................................................
----------------------------------------------------------------------
Ran 90 tests in 0.716s

OK


In [215]:
from lark import Lark, Tree, Token

class Interpreter:
    
    def is_valid(self, relation, universe):
        if not (relation.get_domain() | relation.get_codomain()) <= FiniteSet(*universe):
            raise Exception(f"Relation - {relation} not valid for universe - {universe}!")
    
    def __init__(self, variables=dict(), universe=FiniteSet(), check=False):
        if check == True:
            for value in variables.values():
                self.is_valid(value, universe)
                
        self._variables = variables
        self._universe = FiniteSet(*universe)
        self.parser = Lark('''
                        RELATION: "P".."Z"
                        SET: "A".."O"
                        ?logical_operator: special_operator
                             | logical_operator "and" special_operator  -> conjunction
                             | logical_operator "or"  special_operator  -> disjunction                   
                             | logical_operator "->"  special_operator  -> implication
                             | logical_operator "=="  special_operator  -> equal
                             | logical_operator "!="  special_operator  -> not_equal
                             | logical_operator ">"   special_operator  -> greater_than
                             | logical_operator "<"   special_operator  -> lower_than
                             | logical_operator ">="  special_operator  -> greater_equal
                             | logical_operator "<="  special_operator  -> lower_equal                   
                        ?special_operator: properties
                             | "not" logical_operator      -> negation
                             | product "is" properties     -> is
                             | product "is not" properties -> is_not
                        ?properties: product    
                             | "total_order"   -> total_order
                             | "transitive"    -> transitive
                             | "lattice"       -> lattice
                             | "reflexive"     -> reflexive
                             | "symmetric"     -> symmetric
                             | "asymmetric"    -> asymmetric
                             | "antisymmetric" -> antisymmetric
                        ?product: atom
                             | product "°" atom   -> composition
                             | product "&" atom   -> intersection
                             | product "|" atom   -> union
                             | product "-" atom   -> difference
                             | product "^" atom   -> symmetric_difference
                             | product "*" atom   -> multiplication
                        ?atom: "("logical_operator")"
                             | SET 
                             | RELATION
                        %import common.WS
                        %ignore WS
                 ''', start='logical_operator')
        
    def set_variables(self, variables=dict(), check=False):
        if check == True:
            for value in variables.values():
                self.is_valid(value, self.get_universe())
        self._variables = variables
        
    def get_variables(self, *variables):
        if len(variables) == 0:
            return self._variables
        else:
            selected_variables = dict()
            for variable in variables:
                selected_variables[variable] = self._variables[variable]
            return selected_variables
    
    def add_variable(self, variable, value, check=False):
        if check == True:
            self.is_valid(value, self.get_universe())
        variables = self.get_variables()
        if variable in variables.keys():
            raise Exception(f"Variable - '{variable}' is already defined!")
        variables[variable] = value
        self.set_variables(variables)
        
    def set_variable(self, variable, new_value, check=False):
        if check == True:
            self.is_valid(new_value, self.get_universe())
        variables = self.get_variables()
        if variable not in variables.keys():
            raise Exception(f"Variable - '{variable}' has not been defined yet!")
        variables[variable] = new_value
        self.set_variables(variables)
        
    def set_universe(self, universe=FiniteSet(), check=False):
        if check == True:
            for relation in self.get_variables().values():
                if not (relation.get_domain() | relation.get_codomain()) <= FiniteSet(*universe):
                    raise Exception(f"Universe - {universe} not valid for relation - {relation}!") 
        self._universe = FiniteSet(*universe)
        
    def get_universe(self):
        return self._universe
        
    def constant(self,data):
        try:
            variables = self.get_variables()
            return variables[data]
        except:
            raise Exception(f"Variable - '{data}' doesn't exist!")

    def interpret(self,node):
        if isinstance(node, Tree):
            if node.data == 'negation':
                return not self.interpret(node.children[0])
            elif len(node.children) == 2:
                left, right = node.children  
                if node.data == 'equal':
                    return self.interpret(left) == self.interpret(right)
                elif node.data == 'not_equal':
                    return self.interpret(left) != self.interpret(right)
                elif node.data == 'greater_than':
                    return self.interpret(left) > self.interpret(right)
                elif node.data == 'lower_than':
                    return self.interpret(left) < self.interpret(right)
                elif node.data == 'greater_equal':
                    return self.interpret(left) >= self.interpret(right)
                elif node.data == 'lower_equal':
                    return self.interpret(left) <= self.interpret(right)
                elif node.data == 'conjunction':
                    return self.interpret(left) and self.interpret(right)
                elif node.data == 'disjunction':
                    return self.interpret(left) or self.interpret(right)
                elif node.data == 'implication':
                    return (not self.interpret(left)) or self.interpret(right)
                elif node.data == 'intersection':
                    return self.interpret(left) & self.interpret(right)
                elif node.data == 'union':
                    return self.interpret(left) | self.interpret(right)
                elif node.data == 'difference':
                    return self.interpret(left) - self.interpret(right)
                elif node.data == 'symmetric_difference':
                    return self.interpret(left) ^ self.interpret(right)
                elif node.data == 'multiplication':
                    return self.interpret(left) * self.interpret(right)
                elif node.data == 'composition':
                    return self.interpret(left).composition(self.interpret(right))
                elif node.data == 'is' or node.data == 'is_not':
                    if right.data == 'lattice':
                        return self.interpret(left).is_lattice() if node.data == 'is' else not self.interpret(left).is_lattice()
                    elif right.data == 'total_order':
                        return self.interpret(left).is_total_order() if node.data == 'is' else not self.interpret(left).is_total_order()
                    elif right.data == 'reflexive':
                        return self.interpret(left).is_reflexive() if node.data == 'is' else not self.interpret(left).is_reflexive()
                    elif right.data == 'symmetric':
                        return self.interpret(left).is_symmetric() if node.data == 'is' else not self.interpret(left).is_symmetric()
                    elif right.data == 'asymmetric':
                        return self.interpret(left).is_asymmetric() if node.data == 'is' else not self.interpret(left).is_asymmetric()
                    elif right.data == 'antisymmetric':
                        return self.interpret(left).is_antisymmetric() if node.data == 'is' else not self.interpret(left).is_antisymmetric()
                    elif right.data == 'transitive':
                        return self.interpret(left).is_transitive() if node.data == 'is' else not self.interpret(left).is_transitive()
                else:
                    raise Exception(f"ERROR! {node.data}")
        elif isinstance(node, Token):
            node = self.constant(node.value)
            return node
        else:
            raise Exception(f"ERROR! unkonown instance {type(node)}")

In [216]:
relation1 = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)
relation2 = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)

interpreter = Interpreter({'R':relation1,'S':relation2}, {1,2,3}, True)
#interpreter.set_variables({'R':relation1,'X':HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3,4}, True)}, True) #Error

In [217]:
# abc = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)
# dr = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)

# tree = parser.parse("R is reflexive and S is transitive -> R | S is transitive", {'R':abc, 'S':dr}, {'d','e',1,2,3})

# print(f"[TREE]\n{tree.pretty()}[/TREE]\n")
# print(f"[RESULT -> {interpret(tree)}]")

In [218]:
tree = interpreter.parser.parse("R is transitive and S is transitive and R | S is not transitive")

print(f"[TREE]\n{tree.pretty()}[/TREE]\n")
print(f"[RESULT -> {interpreter.interpret(tree)}]")

[TREE]
conjunction
  conjunction
    is
      R
      transitive
    is
      S
      transitive
  is_not
    union
      R
      S
    transitive
[/TREE]

[RESULT -> False]


In [219]:
tree = interpreter.parser.parse("R is transitive and S is transitive and not R | S is transitive")

print(f"[TREE]\n{tree.pretty()}[/TREE]\n")
print(f"[RESULT -> {interpreter.interpret(tree)}]")

[TREE]
conjunction
  conjunction
    is
      R
      transitive
    is
      S
      transitive
  negation
    is
      union
        R
        S
      transitive
[/TREE]

[RESULT -> False]


In [220]:
U = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)
V = PartialOrder({(1,1),(1,2),(1,3),(2,2),(2,3),(3,3)}, {1,2,3},True)

interpreter.add_variable('U',U, True)
interpreter.add_variable('V',V, True)
tree = interpreter.parser.parse("V is lattice and U|V is transitive ->  V is total_order")

print(f"[TREE]\n{tree.pretty()}[/TREE]\n")
print(f"[RESULT -> {interpreter.interpret(tree)}]")

print(f"\nvariables:\n {interpreter.get_variables()}")

[TREE]
implication
  conjunction
    is
      V
      lattice
    is
      union
        U
        V
      transitive
  is
    V
    total_order
[/TREE]

[RESULT -> True]

variables:
 {'R': 
domain:{1, 2, 3}
codomain:{1, 2, 3}
relation:{(1, 1),(2, 2),(3, 3)}
, 'S': 
domain:{1, 2, 3}
codomain:{1, 2, 3}
relation:{(1, 1),(2, 2),(3, 3)}
, 'U': 
domain:{1, 2, 3}
codomain:{1, 2, 3}
relation:{(1, 1),(2, 2),(3, 3)}
, 'V': 
domain:{1, 2, 3}
codomain:{1, 2, 3}
relation:{(1, 1),(1, 2),(1, 3),(2, 2),(2, 3),(3, 3)}
}


In [224]:
V = PartialOrder({(1,1),(2,2),(3,3)}, {1,2,3},True)

#interpreter.add_variable('V',V) #Error - already defined variable
interpreter.set_variable('V',V,True)
#interpreter.set_universe({1,2},True) #Error - not valid universe

tree = interpreter.parser.parse("V != V ->  S is total_order")




print(f"[TREE]\n{tree.pretty()}[/TREE]\n")
print(f"[RESULT -> {interpreter.interpret(tree)}]")

print(f"\nvariables:\n {interpreter.get_variables('V','S')}")

[TREE]
implication
  not_equal
    V
    V
  is
    S
    total_order
[/TREE]

[RESULT -> True]

variables:
 {'V': 
domain:{1, 2, 3}
codomain:{1, 2, 3}
relation:{(1, 1),(2, 2),(3, 3)}
, 'S': 
domain:{1, 2, 3}
codomain:{1, 2, 3}
relation:{(1, 1),(2, 2),(3, 3)}
}


In [225]:
interpreter.get_variables()

{'R': 
 domain:{1, 2, 3}
 codomain:{1, 2, 3}
 relation:{(1, 1),(2, 2),(3, 3)},
 'S': 
 domain:{1, 2, 3}
 codomain:{1, 2, 3}
 relation:{(1, 1),(2, 2),(3, 3)},
 'U': 
 domain:{1, 2, 3}
 codomain:{1, 2, 3}
 relation:{(1, 1),(2, 2),(3, 3)},
 'V': 
 domain:{1, 2, 3}
 codomain:{1, 2, 3}
 relation:{(1, 1),(2, 2),(3, 3)}}

In [226]:
interpreter.get_universe()

FiniteSet(1, 2, 3)